# Pytorch를 활용한 hand sign 검출

In [4]:
import pandas as pd
import numpy as np
%matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import random
import os
import time
import copy
import glob
from tqdm import tqdm as tqdmd

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as transforms
from torchvision.io import read_image
import torchvision.models as models
from torchsummary import summary


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Using matplotlib backend: Qt5Agg


## Dataset Review

![ASL](./image/01.png)
* ASL(American Sign Language) alphabet은 총 26개가 있지만 dataset에서는 25개 밖에 없다. 
* 그 이유는 Z가 검지로 Z를 그리는 것이기 때문에 사진으로는 표현이 불가능.

## Image dir, path

In [17]:
TRAIN_DIR = "Train/"
TEST_DIR = "Test/"
TRAIN_DIR_LIST = os.listdir(TRAIN_DIR)
TEST_DIR_LIST = os.listdir(TEST_DIR)

In [71]:
#Trian Dataset
df = pd.DataFrame(columns = ['label', 'dir'])
for i in TRAIN_DIR_LIST:
    for j in os.listdir(TRAIN_DIR + i +"/"):
        data = {'label' : i, 'dir' : TRAIN_DIR + i + "/" + j }
        df = df.append(data, ignore_index=True)
df.to_csv("Train/train.csv")

In [72]:
df = pd.DataFrame(columns = ['label', 'dir'])
for i in TEST_DIR_LIST:
    for j in os.listdir(TEST_DIR + i +"/"):
        data = {'label' : i, 'dir' : TEST_DIR + i + "/" + j }
        df = df.append(data, ignore_index=True)
df.to_csv("Test/test.csv")

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7172 entries, 0 to 7171
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   7172 non-null   object
 1   dir     7172 non-null   object
dtypes: object(2)
memory usage: 112.2+ KB


In [74]:
train_dataset = pd.read_csv("Train/train.csv", index_col = 0)
test_dataset = pd.read_csv("Test/test.csv", index_col = 0)

In [75]:
train_dataset.head(5)

,label,dir
0,A,Train/A/10014_A.jpg
1,A,Train/A/10021_A.jpg
2,A,Train/A/10023_A.jpg
3,A,Train/A/10050_A.jpg
4,A,Train/A/10056_A.jpg


## Data Check

In [83]:
train_dataset.loc[0][0]

'A'

In [77]:
image = Image.open(train_dataset.loc[0][1])

In [85]:
image.size

(28, 28)

In [89]:
image

## Mapping
* A ~ Y -> 0 ~ 24

In [102]:
mapping = {}
for i in range(25):
    mapping[chr(65+i)] = i

In [106]:
train_dataset = train_dataset.replace(mapping)
test_dataset = test_dataset.replace(mapping)

In [107]:
train_dataset.value_counts()

label  dir                
0      Train/A/10014_A.jpg    1
17     Train/R/15735_R.jpg    1
       Train/R/15677_R.jpg    1
       Train/R/15674_R.jpg    1
       Train/R/15670_R.jpg    1
                             ..
8      Train/I/18798_I.jpg    1
       Train/I/18759_I.jpg    1
       Train/I/18745_I.jpg    1
       Train/I/1873_I.jpg     1
24     Train/Y/9997_Y.jpg     1
Length: 27455, dtype: int64

## transformer
* RandomHorizontalFlip(p)


Horizontally flip the given image randomly with a given probability.
* RandomRotation(degrees)


Rotate the image by angle.
* RandomVerticalFlip(p)


Vertically flip the given image randomly with a given probability.



In [87]:
DIM = (28, 28)
NUM_CLASSES = 25
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

In [88]:
def transforms(value = 'val'):
    if value == 'train':
            transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(DIM),
            transforms.RandomHorizontalFlip(p = 0.5),
            transforms.RandomRotation(90),
            transforms.RandomVerticalFlip(p = 0.5),
            transforms.ToTensor(),
            transforms.Normalize(MEAN, STD)
        ])
            return transform
    
    elif value == 'val':
            transform = torchvision.transforms.Compose([
            torchvision.transforms.ToPILImage(),
            torchvision.transforms.Resize(DIM),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(MEAN, STD)
        ])
            return transform

## DataLoader

In [ ]:
class CassavaDataset(Dataset):
    
    def __init__(self, img_ids, labels, folder_dir = 'tran_images/', 
                 transform=None):
        self.img_ids = img_ids
        self.labels = labels
        self.folder_dir = folder_dir
        self.transform = transform
        
    
        
    def __len__(self):
        return len(self.img_ids)
    
    def __getitem__(self, idx):
        # 이미지 read 및 label 저장
        # cat
        img = read_image(os.path.join(self.folder_dir+self.img_ids[idx]))

        
        if self.transform:
            img = self.transform(img)



        label = torch.tensor(self.labels[idx], dtype = torch.long)
        # image, label을 return
        return img, label